## Importing libraries and data

In [2]:
# Importing libraries
import pandas as pd

In [3]:
# Reading .csv
data = pd.read_csv(r'Data/CSV/1-AV-001-PR.csv', sep=';')
data.head()

# Dropping rows with NaN for facies thickness -- TODO

,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion
0,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"1,76",NaN,NaN
1,1-AV-001-PR,Paranapanema,Simple lava (basic | massive interior),"2,16","2,16",NaN
2,1-AV-001-PR,Paranapanema,Simple lava (basic | L. crust),"4,48","13,36","0,616766467"
3,1-AV-001-PR,Paranapanema,Siliciclastics,"8,24",NaN,NaN
4,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"0,64",NaN,NaN


## Flow Counter Algorithm

In [6]:
# Flow Counter v2 -- Accounting for Unit Intercalation


# Setting up variables
flow_counter = 0
flow_list = []
flow_dict = {}

# Algorithm
for i in range(len(data)):
    if i==0: # Setting up the first iteration]
        flow_counter += 1
        flow_dict[data['Facies'].iloc[i]] = flow_counter
        flow_list.append(flow_counter)
    elif i == (len(data)-1): # Checking for the end of the list (working around out-of-bounds bug)
        flow_list.append(flow_counter)
        break
    elif data['Facies'].iloc[i] == "Siliciclastics" or data['Facies'].iloc[i] == "Volcaniclastics (undiff.)": # Check for siliciclastic OR volcaniclastic; append a "-" for these units:
        flow_list.append('-')        
    else: # If we're indeed dealing with lava flow data  
        if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before
            if data['Facies'].iloc[i] == 'Simple lava (basic | L. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)': # Check for Simple lower crust OR compound flows:
                flow_list.append(flow_counter)
                flow_counter += 1
            else:
                flow_list.append(flow_counter)
        else: # If Unit changes
            if data['Unit'].iloc[i] in flow_dict: # If unit has already been added as a key to the flow_dict
                flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                flow_counter = flow_dict[data['Unit'].iloc[i]]
                flow_list.append(flow_counter)
            else: # If unit is NOT already associated with a value in the flow_dict
                flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                flow_counter = 1
                flow_dict[data['Unit'].iloc[i]] = flow_counter
                flow_list.append(flow_counter)
                
print(flow_dict)
print(flow_list)

{'Simple lava (basic | U. crust)': 1, 'Paranapanema': 33, 'Pitanga': 34, 'Vale do Sol': 1}
[1, 1, 1, '-', 2, 2, 2, '-', 3, 3, 3, '-', 4, 4, 4, 5, '-', 6, 7, 7, 7, 8, 9, 9, 9, '-', 10, 10, 10, '-', 11, 11, 11, '-', 12, 12, 12, '-', 13, 13, 13, 14, 14, 14, '-', 15, 16, 16, 16, 17, 18, 18, 18, '-', 19, 19, 19, '-', 20, 20, 20, 21, '-', 22, 22, 22, '-', 23, 24, 24, 24, '-', 25, 25, 25, 26, 26, 26, '-', 27, 27, 27, 1, 1, 1, 2, 3, 3, 3, 4, 28, 28, 28, '-', 29, 30, 30, 30, 31, 32, 32, 32, '-', 5, '-', 5, 6, 6, 6, '-', 7, 8, 8, 8, 9, 10, 11, 11, 11, '-', 12, 12, 12, '-', 13, 13, 13, 14, 14, 14, '-', 15, 16, 16, 16, 17, 17, 17, 18, '-', 19, 19, 19, 20, 20, 20, 21, 21, 21, '-', 22, 22, 22, 23, 24, 24, 24, '-', 25, 25, 25, 26, 26, 26, '-', 27, 28, 28, 28, 29, 29, 29, '-', 30, 30, 30, 31, 32, 32, 32, '-', 33, 33, 33, 1, 1, 1]


## Putting the algorithm inside a function for better functionality

## Others /// Notes: